# Triathlon Dataset: Cleaning
Several tasks are completed to clean the data before any analysis can be performed.
 - Merging tables, removing columns and renaming headers
 - Locations and nationality abbreviations changed to full names
 - Athlete gender and age groups inconsistencies
 - Converting time data to integer number of seconds
 - Testing for times outside of world records or cutoffs

In [1]:
import pandas as pd
from datetime import datetime as dt
import numpy as np
import os

## Importing and merging the data
The two csv files, corresponding to the events and the athlete results, are read in. As long as these files are in the same folder as the Python file, the data will be read in correctly. The two files are merged to create a single table following the reading of the csv files.

In [2]:
cwd = os.getcwd() #Finds the current working directory. Allows data to be read without changing the filepath.
df_events = pd.read_csv(cwd+"\postgres_public_tristat_events.csv")
df_stat = pd.read_csv(cwd+"\postgres_public_tristat_stat.csv")

#Renaming the columns before merging the dataframes.
df_stat.columns = ['event_link', 'gender', 'person_link', 'person_flag', 'person_name',
       'age_group', 'swim_time', 't1_time', 'cycle_time', 't2_time', 'run_time', 'finish_time']
df_events.columns = ['date', 'location', 'event', 'f_count', 'm_count', 'event_link']

#An inner join to merge the two tables. 
df_full = df_events.merge(df_stat, how='inner', on='event_link')

## New columns and removed records
Continuing to alter the data to allow for the cleaning and analysis to come.

In [3]:
#New columns added to help to clean, sort and analyse the data. These are the event links split into individual columns.
event_distances, organisers, years, links = [[], [], [], []]

for i in df_full['event_link']: #Iterating through the event_link series
    years.append(i.split('/')[-1]) #Year of the race
    event_distances.append(i.split('/')[-2]) #Selects the race distance and appends to list for every event
    organisers.append(i.split('/')[-4]) #Organiser info

df_full['year'] = years
df_full['year'] = pd.to_datetime(df_full['year']).dt.year
df_full['distance'] = event_distances #New column identifies race distance
df_full['organiser'] = organisers

In [4]:
#Only wish to have Ironman events of full and half distance.
#Remove all other records.
df_iron = df_full[(df_full.organiser=='ironman')&((df_full.distance=='half')|(df_full.distance=='full'))].copy()

#Removes several columns: date, f_count, m_count, event_link, person_name, organiser.
df_iron = df_iron[['event', 'location', 'year', 'distance', 'person_link', 'gender', 'person_flag', 'age_group', 'swim_time', 't1_time', 'cycle_time', 't2_time', 'run_time', 'finish_time']].copy()

records_removed = df_full.shape[0] - df_iron.shape[0]
print(f'Number of records removed: {records_removed}')

Number of records removed: 643861


## Unique athlete identifier
Cleaning inconsistencies in the athlete information would be easiest if there is an identifier for each athlete. The person_link column appears to provide this. This would allow me to homogenise columns such as gender and nationality for each athlete. However, the person_link is not unique for each athlete. There are several such cases which appear to confirm this.
 - Athletes with the same name and nationality have the same person_link.
 - Some of the most common person_link values appear to be placeholders for unknown names.

In [5]:
#This person_link corresponds to two different athletes.
#My father has completed Ironman UK 2019, but not the other event.
df_full[df_full['person_link']=='/gbr/profile/cooke-richard']

,date,location,event,f_count,m_count,event_link,gender,person_link,person_flag,person_name,age_group,swim_time,t1_time,cycle_time,t2_time,run_time,finish_time,year,distance,organiser
609282,2019-07-14,GBR,Ironman UK 2019,173,1166,/rus/result/ironman/uk/full/2019,M,/gbr/profile/cooke-richard,GBR,"Cooke, Richard",M40-44,1:13:00,9:30,7:49:53,10:09,4:42:34,14:05:06,2019,full,ironman
1719878,2015-06-14,GBR,Ironman Staffordshire 70.3 2015,384,1528,/rus/result/ironman/staffordshire/half/2015,M,/gbr/profile/cooke-richard,GBR,"Cooke, Richard",M35-39,35:18,6:58,3:10:22,4:10,1:50:31,5:47:19,2015,half,ironman


In [6]:
#The most common person_link values. 'triathlete-taiwan-2' probably isn't an individual person.
df_full['person_link'].value_counts().head()

/tpe/profile/triathlete-taiwan-2    255
/jpn/profile/ueda-ai                207
/ukr/profile/elistratova-yuliya     175
/jpn/profile/hosoda-yuichi          171
/ukr/profile/sapunov-danylo         168
Name: person_link, dtype: int64

## Replacing abbreviated locations/nationalities

During my analysis, it was revealed that the abbreviated country names are unclear for all except the most common nations. I would like to replace the abbreviations with full names for the countries. I have used the country_converter package to do this.

In [7]:
#Importing the package and setting up a method to allow for the codes to be converted to names.
import country_converter as coco
cc = coco.CountryConverter()

In [8]:
df_countries = df_iron.copy()

nationalities = list(df_countries['person_flag'].unique()) #List of unique country codes.
nationalities = [x for x in nationalities if str(x) != 'nan'] #Remove the null value in the list. Causes an issue during the use of country_converter.

In [9]:
country_conversion_dict = {} #Dictionary to be populated, then used to clean my data.

#Iterate through the codes, converting one at a time
for i in nationalities:
    short_name = cc.convert(names=[i], src='IOC', to='name_short', not_found=None) #Converts based on the IOC codes (Olympic countries only)
    if short_name == i:
        short_name = cc.convert(names=[i], src='Eora', to='name_short', not_found=None) #Converts based on Eora - a global supply chain database
    
    country_conversion_dict[i]=short_name #Key-value pairs for conversion added

MAF not found in IOC
MTQ not found in IOC
NCL not found in IOC
PYF not found in IOC
IMN not found in IOC
CCK not found in IOC
CCK not found in Eora
CUW not found in IOC
CUW not found in Eora
GGY not found in IOC
JEY not found in IOC
NIU not found in IOC
REU not found in IOC
GLP not found in IOC
GUF not found in IOC
HMD not found in IOC
HMD not found in Eora
GIB not found in IOC
FRO not found in IOC
SPM not found in IOC
BLM not found in IOC
MAC not found in IOC
XKX not found in IOC
XKX not found in Eora
ATA not found in IOC
ATA not found in Eora
ALA not found in IOC
GRL not found in IOC
FLK not found in IOC
SJM not found in IOC
IOT not found in IOC
IOT not found in Eora
CXR not found in IOC
CXR not found in Eora
MSR not found in IOC
PCN not found in IOC
SGS not found in IOC
SGS not found in Eora
BVT not found in IOC
BVT not found in Eora
UMI not found in IOC
UMI not found in Eora
NFK not found in IOC
SXM not found in IOC
SXM not found in Eora
MYT not found in IOC
MNP not found in IOC
SH

In [10]:
#Some codes filled manually. Tends to be groups of islands or Antarctic territories belonging to European nations/USA.
country_conversion_dict.update({
    'CCK':'Cocos (Keeling) Islands', 'CUW':'Curacao', 'HMD':'Heard Island and McDonald Islands', 'XKX':'Kosovo', 'ATA':'Antarctica',
    'IOT':'British Indian Ocean Territory', 'CXR':'Christmas Island', 'SGS':'South Georgia and the South Sandwich Islands', 'BVT':'Bouvet Island', 'UMI':'United States Minor Outlying Islands',
    'SXM':'Sint Maarten', 'ATF':'French Southern and Antarctic Territories', 'BES':'Bonaire, Saint Eustatius and Saba', 'SCG':'Serbia and Montenegro', 'AHO':'Netherlands Antilles'
})

#This converts the abbreviations to the full names for both columns.
df_countries.replace({'person_flag': country_conversion_dict, 'location': country_conversion_dict}, inplace=True)

## Removing events with missing nationalities

In [11]:
#Counts number of records with null person_flag values per event.
df_missing_nationalities_by_event = df_countries[df_countries.person_flag.isnull()][['event', 'location']].groupby(by='event').count()

#Calculate proportion of missing values to decide whether to remove events from the dataset.
competitors_by_event = df_countries.groupby('event').count()['location'] #Counts number of participants per event
df_missing_nationalities_by_event = df_missing_nationalities_by_event.merge(competitors_by_event, how='inner', on='event')
df_missing_nationalities_by_event.columns = ['number_missing', 'total_competitors']

#Proportions will be used to highlight events that haven't recorded the nationalities.
df_missing_nationalities_by_event['proportion'] = df_missing_nationalities_by_event['number_missing']/df_missing_nationalities_by_event['total_competitors']

events_to_remove = df_missing_nationalities_by_event[df_missing_nationalities_by_event['proportion']>0.1].index #Seems like 0.1 is a good compromise for keeping most data, removing most missing values.

df_clean_countries = df_countries[~df_countries.event.isin(events_to_remove)] #Events are removed.

## Cleaning the age_group column

There are a set of standard Ironman age groups. Additionally, there are age groups designating relay teams and disability categories. There are also several instances where the age group has been mis-recorded. I have decided to retain the competitors with standard age groups. This will simplify the analysis by reducing the number of categories whie also cleaning the data.

In [12]:
df_age = df_clean_countries.copy()
df_age['age_group'].value_counts()

M40-44       330356
M35-39       311927
M45-49       263551
M30-34       256918
M50-54       174098
              ...  
M5                1
MTBC              1
MINMEMORY         1
M65+              1
M90+              1
Name: age_group, Length: 129, dtype: int64

In [13]:
#The age group boundaries according to the Ironman website.
iron_groups = ['F18-24','F25-29','F30-34','F35-39','F40-44','F45-49','F50-54','F55-59','F60-64','F65-69','F70-74','F75-79','F80-84','F85-89','F90+',
          'M18-24','M25-29','M30-34','M35-39','M40-44','M45-49','M50-54','M55-59','M60-64','M65-69','M70-74','M75-79','M80-84','M85-89','M90+',
          'MPRO','FPRO']

#Any other age groups found in the dataset.
other_groups = ['MPC','FPC','PC','MST','FST','M','F','MRELAY','FRELAY','MEX','FNKNOWN','M20-24','F20-24',
                'M17-18','FJ19','M15-16','MTEAM','FTEAM','M15-19','RELAY','F15-19','MCM18-24','MCM25-29','FCF18-24',
                'MU6','M18-25','M80+','FPARA','MCOL','FCOL','MNKNOWN','M18','M25','M30','M35','M45','M40','F25','M50',
                'F18','M55','F30','F40','F35','F45','M60','F50','M70','F55','M65','F65','F60','MPRO30-34','MPRO40-44',
                'MPRO25-29','MPRO35-39','FPRO30-34','FPRO35-39','M0-44','MAWAD','ML-Z','FRELAYCOED','MRELAYCOED',
                'MCLYDESDALES','MAQUABIKE','FATHENAS','MPSL-Z','MINMEMORY','FPROILIPINOPRO','M65+','M19UND','F19UND',
                'M0','MNOAGE','MCLY','FCLY','MTBC','M5','M54','F44','M4','M9','F75+','M70+','M70-','MIXED','MHERREN',
                'MFIR','FCOUPLES','MCOUPLES','MDA','40-44','U','MX-RLY','MCLY-U','M-RLY','FX-RLY','F-RLY','FATH-U',
                'MCLY-O','FATH-O','M75-99','MPROEN','MFPROO','MILITARY','M7-9','F7-9','50-54','MHANDCYCLE','45-49',
                '35-39','30-34','55-59','M10-14','60-64','25-29','M70-99','15-19','20-24','70-74','F90-94','65-69',
                '75-79','F30-44','M25-30','M55-60','F60-6','F45-50']


standard_group = df_age[df_age['age_group'].isin(iron_groups)]['age_group'].size #Number athletes with the standard groups
other_group = correct_group = df_age[df_age['age_group'].isin(other_groups)]['age_group'].size #Number with non-standard groups

print(f'There are {standard_group} athletes with standard age grouping; {other_group} with non-standard age grouping.')

There are 2196844 athletes with standard age grouping; 31603 with non-standard age grouping.


In [14]:
#Calculating proportions of competitors at each event with the standard age groups.
test = df_age.copy()

proportions = []

for i in test['event'].unique():
    event = test[test.event==i] #Dataframe containing the records for a single event
    iron = event[event['age_group'].isin(iron_groups)]['age_group'].size #Number records with standard age groups
    non_iron = event[event['age_group'].isin(other_groups)]['age_group'].size #Number with non-standard groups
    proportions.append([i, iron/(iron+non_iron)]) #Append event name and the proportion to be analysed later
    
proportions[:2] #Both Florida and NZ have the correct age groups!    

[['Ironman Florida 70.3 2022', 0.9993279569892473],
 ['Ironman New Zealand 2022', 1.0]]

In [15]:
#Finding the events with high proportions of non-standard age groups.
events_bad_groups = []

for i in range(len(proportions)):
    if proportions[i][1] < 0.1: #Increasing to 0.9 only yields an extra 2k entrants with non-standard age groups.
        events_bad_groups.append(proportions[i][0])
        
#Removing all records pertaining to the untrustworthy events.
df_events_removed = df_age[~df_age['event'].isin(events_bad_groups)].copy()

#Removing the remainder of the records with the non-standard age groups.
df_clean_age = df_events_removed[df_events_removed['age_group'].isin(iron_groups)].copy()

## Cleaning the gender column

### Attempt 1: Standardising gender by person_link
This method relied on a unique person_link for each athlete, by converting all values of gender to be the same for each. However, since the person_link is not a unique identifier, this technique is flawed.

In [16]:
from collections import Counter

def gender_string_check(s):
    '''A function used to check if all of an athlete's recorded genders match.
    Allow these athletes to be removed from the gender-correcting analysis.'''
    
    if s == len(s) * s[0]:
        return True
    else:
        return False

#Passing the dataframe of results into this function returns another dataframe with a cleaned column.
#Records are removed if there is no discernible mode gender for each person link.
def gender_correct(x):
    '''Performs the gender corrections. Where a majority gender is recorded, all values are changed to this gender.
    If there is no majority, gender values replaced with a NaN so that records can be removed.
    '''
    
    df_temp = x[['person_link', 'gender']].copy()
    df_temp = df_temp.groupby(by='person_link').sum() #Creates string of all gender values combined eg FFFFFMFF
    df_temp.drop(df_temp[df_temp.gender.map(gender_string_check) == True].index, inplace=True) #Removes records with consistent gender, eg M or FFF
    
    gender_update = x.copy()
    for i in df_temp.index: #Loop through every athlete with inconsistent gender records
        count = Counter(df_temp.loc[i, 'gender']) #Returns dictionary with count of each M/F value eg {'M': 5, 'F': 3}

        if count['F'] == count['M']: #Cannot tell which is the likely correct gender
            gender_update.loc[gender_update[gender_update['person_link']==i].index, 'gender'] = np.nan
        elif count['F'] > count['M']: #Change all to 'F'
            gender_update.loc[gender_update[gender_update['person_link']==i].index, 'gender'] = 'F'
        else: #Change all to 'M'
            gender_update.loc[gender_update[gender_update['person_link']==i].index, 'gender'] = 'M'
    
    
    return gender_update

### Attempt 2: Comparing gender and age group

- Can make sure that the gender specified in the gender column matches the one specified by the age group.
- There are 249 records which have non-matching genders. I will just remove them. It is both too many to go through manually and not enough to be concerned about losing too much data.

In [17]:
df_gender = df_clean_age.copy() #Taking forward the most clean version of the data.

df_gender.reset_index(inplace=True, drop=True)

In [18]:
df_gender_age = df_gender[['gender', 'age_group']]

#Aiming to create a new column which has the age group gender extracted.
age_genders = []

for i in df_gender['age_group']: #Iterating through the event_link series
    age_genders.append(str(i)[0]) #Gender from age group
    
df_gender['age_gender'] = age_genders

df_gender[df_gender['gender'] != df_gender['age_gender']][['person_link', 'gender', 'age_group']]

,person_link,gender,age_group
6759,/aus/profile/williamson-stuart,M,F35-39
16361,/phi/profile/guz-dela-rosa,F,M35-39
17905,/usa/profile/lazarus-andre,F,M18-24
27947,/swe/profile/gerremo-hans,F,M70-74
29207,/pan/profile/aued-lysa,M,F40-44
...,...,...,...
2191398,/ger/profile/apel-stefanie,M,F25-29
2191442,/usa/profile/castaneda-angelika,M,F50-54
2191459,/sui/profile/wieneke-silvia,M,F40-44
2192637,/ger/profile/hauhann-katrin,M,F25-29


In [19]:
#Are there any races particularly guilty of poor administration?
df_gender[df_gender['gender'] != df_gender['age_gender']][['event']].value_counts()

#Top offenders do not have enough mistakes to worry about the whole race. At worst it is 1/50 records from the Durban race.

event                                   
Ironman Durban 70.3 2021                    17
Ironman UK 2006                              7
Ironman UK 2007                              6
Ironman Europe 1994                          5
Ironman Dubai 70.3 2021                      4
                                            ..
Ironman Japan 2007                           1
Ironman Indian Wells La Quinta 70.3 2022     1
Ironman Indian Wells La Quinta 70.3 2019     1
Ironman Haugesund Norway 70.3 2018           1
Ironman Zell am See-Kaprun 70.3 2022         1
Length: 138, dtype: int64

In [20]:
df_clean_gender = df_gender[df_gender['gender'] == df_gender['age_gender']].copy()

## Converting the time columns to seconds values

In [21]:
def time_function(x):
    '''Convert a time string into an integer number of seconds. 
    Accepts inputs in the M:SS, MM:SS, H:MM:SS and HH:MM:SS formats.'''
    
    #Adds minutes and hours values so that all times have hours, minutes and seconds components.
    if len(x)==5:
        x = '0:'+x #Adds an hours components to MM:SS times.
    elif len(x)==4:
        x = '0:0'+x #Adds an hours and second minutes digit to M:SS times.

    date_format = '%H:%M:%S'
    time = dt.strptime(x, date_format) #Converts time string to datetime format.
    time = time.second + time.minute*60 + time.hour*3600 #Calculates integer number of seconds.
    
    return time

In [22]:
df_time = df_clean_gender.copy()
df_time.drop(['age_gender'], axis=1, inplace=True)

#Applies the time function to every time-based column in the dataframe.
df_time.iloc[:, 8:] = df_time.iloc[:, 8:].applymap(lambda x:time_function(x)) #Assumes that the columns are the rightmost set.

## Cleaning the time columns
There are two potential problems with the time columns
 - Missing data (cancelled swims, no transition times, etc)
 - Unrealistic times (quicker than record pace, slower than cutoff times)

In [23]:
#Every record has a finish time, but other calues are missing.
print('Number of missing values per column:')
df_time[['swim_time', 't1_time', 'cycle_time', 't2_time', 'run_time', 'finish_time']].isin([0]).sum(axis=0)

Number of missing values per column:


swim_time       62392
t1_time        139291
cycle_time      17230
t2_time         40643
run_time         3256
finish_time         0
dtype: int64

In [24]:
#Finding the events which have full time information (non-zero time for all 6 columns)
test = df_time.copy()

event_indicators = []

for i in test['event'].unique(): #Iterate through the events
    df_event = test[test['event']==i][['event', 'swim_time', 't1_time', 'cycle_time', 't2_time', 'run_time', 'finish_time']]
    column_zeros = df_event.isin([0]).sum(axis=0) #Number of zeros per column calculated for each event
    
    #This loop returns a True value for the indicator if there are no missing times. False is there's a missing value.
    for j in column_zeros:
        if j==0:
            indicator=True
        else:
            indicator=False
            break
            
    proportion_zeros = column_zeros.sum()/df_event.iloc[:, 1:].size
    
    #The indicator reveals whether there is any missing data; the proportion reveals how much missing data.
    event_indicators.append([i, indicator, proportion_zeros]) 

In [25]:
#Some summary statistics on the metrics calculated in the last cell
events_missing_times, events_all_times, events_nearly_complete = [[], [], []]

for i in event_indicators:
    if i[1] == True:
        events_all_times.append(i[0]) #Events with no missing times
    else:
        events_missing_times.append(i[0]) #Events with any amount of information.
    
    #0.01 is chosen here because it provides good value for number of events, without having too much missing data.
    #0.001 has 556 events - lots less events.
    #0.1 has 990 events - a very small increase with a large missing data penalty.
    if i[2]!=0 and i[2]<0.01: 
        events_nearly_complete.append(i[0]) #Events with less than 1% missing information.

print(f'There are {len(events_all_times)} events with full timing information.')
print(f'{len(events_missing_times)} events have at least one missing time.')
print(f'Of these, {len(events_nearly_complete)} have less than 1% missing times.')

There are 489 events with full timing information.
1089 events have at least one missing time.
Of these, 929 have less than 1% missing times.


In [26]:
#Dataframe containing the events with complete time information.
df_complete = df_time[df_time['event'].isin(events_all_times)].copy()

#Dataframe containing the events with less than 1% of missing time information.
df_nearly_complete = df_time[df_time['event'].isin(events_nearly_complete)].copy()

#Can have triple the amount of data when including the nearly-complete events.
print(f'{df_complete.shape[0]} records from complete events. {df_nearly_complete.shape[0]} records from nearly complete events.')

630744 records from complete events. 1373055 records from nearly complete events.


In [27]:
#Removing all records with the zeros from df_nearly_complete (around 4500 records).
df_nearly_complete = df_nearly_complete[(df_nearly_complete['swim_time'] != 0)&(df_nearly_complete['t1_time'] != 0)&(df_nearly_complete['cycle_time'] != 0)&(df_nearly_complete.t2_time != 0)&(df_nearly_complete['run_time'] != 0)]

## Divergence of the data
I will now take forward two dataframes:
- All records, regardless of completeness of time column data.
- A dataframe with no missing time data. Some events and records will be removed to achieve this.

These two dataframes will be used for different purposes when analysing and visualising the data. This will depend only on whether the completeness of time data is required for the analysis in question. For example, calculating an average swim time for full distance Ironman events would be skewed by the presence of missing data.

In [28]:
df_all_data = df_time.copy()

df_complete_time_data = pd.concat([df_complete, df_nearly_complete], ignore_index=True)

## Incorrect time data
I will set upper and lower bounds on each of the timing splits. This will identify times which are quicker than the world records or slower than the cutoff times. Any records flagged in this way be removed, and events with larger proportions of 'incorrect' times can be identified.

In [29]:
def time_test(x):
    '''A function which performs checks on the time columns of the triathlon data.
    
    Keyword arguments:
    x -- triathlon results dataframe.
    
    Returns:
    df_indicators -- a dataframe containing True/False values, indicating whether splits are within the accepted range.
    '''
    x.reset_index(inplace=True, drop=True) #Ensures that iterating through the rows is successful
        
    split_indicators = []
    
    #Iterate through the rows to check the time for its position between the boundaries.
    for i in x.index:
        #First assign a set of time boundaries based on the race distance of the record
        if x.loc[i, 'distance'] == 'full':
            swim_low_time , swim_cut_time , bike_low_time , bike_cut_time , run_low_time , finish_low_time , finish_cut_time = [2400, 8400, 14400, 37800, 9000, 27600, 61200]
        elif x.loc[i, 'distance'] == 'half':
            swim_low_time , swim_cut_time , bike_low_time , bike_cut_time , run_low_time , finish_low_time , finish_cut_time = [1300, 4200, 6900, 19800, 4080, 12360, 30600]

        #Setting up some variables which are assigned the True/False values.
        swim_low, swim_cut, bike_low, bike_cut, run_low, finish_low, finish_cut = [False, False, False, False, False, False, False]
        event = x.loc[i, 'event']

        #A series of tests which check whether the splits are within the expected bounds.
        #If a time exceeds a cutoff, or is less than a record, the relevant variable is assigned a True value.
        if x.loc[i, 'swim_time'] < swim_low_time:
            swim_low = True
        if x.loc[i, 'swim_time'] > swim_cut_time:
            swim_cut = True
        if x.loc[i, 'cycle_time'] < bike_low_time:
            bike_low = True
        if (x.loc[i, 'cycle_time']+x.loc[i, 'swim_time']) > bike_cut_time: 
            bike_cut = True
        if x.loc[i, 'run_time'] < run_low_time:
            run_low = True
        if x.loc[i, 'finish_time'] < finish_low_time:
            finish_low = True
        if x.loc[i, 'finish_time'] > finish_cut_time:
            finish_cut = True

        #Append the index, event name and indicators to a list
        split_indicators.append([i, event, swim_low, swim_cut, bike_low, bike_cut, run_low, finish_low, finish_cut])
    
    #Turning the list of list into a dataframe for further analysis, before returning this dataframe.
    time_indicators = pd.DataFrame(split_indicators, columns=['index', 'event', 'swim_low', 'swim_cut', 'bike_low', 'bike_cut', 'run_low', 'finish_low', 'finish_cut'])
    time_indicators.set_index('index', inplace=True)
    
    return time_indicators

In [30]:
df_time_indicators = time_test(df_complete_time_data)

In [31]:
#Some exploratory measures of the results.

print('Number of times flagged per measure. Mostly faster than expected splits, rather than missed cut offs.')
bad_times_by_boundary = df_time_indicators[['swim_low', 'swim_cut', 'bike_low', 'bike_cut', 'run_low', 'finish_low', 'finish_cut']].sum()
display(bad_times_by_boundary)

print('Events with large numbers of times flagged.')
bad_times_by_event = df_time_indicators.groupby('event').sum().sum(axis=1).sort_values(ascending=False).head(10)
display(bad_times_by_event)

Number of times flagged per measure. Mostly faster than expected splits, rather than missed cut offs.


swim_low      30306
swim_cut       4636
bike_low       8254
bike_cut       1438
run_low        3555
finish_low     3625
finish_cut     6075
dtype: int64

Events with large numbers of times flagged.


event
Ironman New Zealand 2012         4795
Ironman Muncie 70.3 2012         3831
Ironman Des Moines 70.3 2021     2452
Ironman Louisville 2018          1942
Ironman Boise 70.3 2012          1809
Ironman Santa Cruz 70.3 2017     1761
Ironman Connecticut 70.3 2019    1708
Ironman Chattanooga 70.3 2019    1663
Ironman Melbourne 2013           1644
Ironman Barcelona 2021           1497
dtype: int64

## Cleaning based on the time test

A 1% of incorrect times within an event will again be used as a threshold to keep that event in the dataset taken forward for analysis. Once the untrustworthy events are removed, the remaining records with incorrect times are removed individually.

In [32]:
#Looking into the proportions of participants which have suspect times for each event.
#Any split which has >1% of flagged times will results in the event being excluded from the timing analysis.

#Calculating proportions of incorrect/unlikely times.
event_indicator_sums = df_time_indicators.groupby('event').sum() #Number of violations by event by split
competitors_by_event = df_complete_time_data.groupby('event').count()['location'] #Counts number of participants per event
df_event_indicators = event_indicator_sums.merge(competitors_by_event, how='inner', on='event') #Combines indicators df with the participants series
df_event_indicators.columns = ['swim_low', 'swim_cut', 'bike_low', 'bike_cut', 'run_low', 'finish_low', 'finish_cut', 'participants'] #Renames columns
df_event_indicators_percentage = df_event_indicators[['swim_low', 'swim_cut', 'bike_low', 'bike_cut', 'run_low', 'finish_low', 'finish_cut']].div(df_event_indicators.participants, axis=0) #Divides summed indicators by number of participants

#Now assess which events have high proportions of false timings. 
events_to_drop = []

#Checks for any proportions >=1% in every event, appending the events to a list.
for i in df_event_indicators_percentage.index: #Iterate through the events
    if df_event_indicators_percentage.loc[i].ge(0.01).any(): #Find events with any row greater than 0.01
        events_to_drop.append(i) #Append these events to a list.
        
print(f'There are {len(events_to_drop)} events with >1% of bad times')

There are 274 events with >1% of bad times


In [33]:
#Ironman New Zealand 2012 is a half distance event, incorrectly labelled.
df_nz_2012 = df_complete_time_data[df_complete_time_data['event']=='Ironman New Zealand 2012']

#Indicators with full race distance
df_nz_2012_indicators = time_test(df_nz_2012)
display(df_nz_2012_indicators.sum())

#Change to half distace labels
df_nz_2012 = df_nz_2012.assign(distance = 'half', event = 'Ironman New Zealand 70.3 2012')

#Indicators with half race distance
df_nz_2012_indicators = time_test(df_nz_2012)
display(df_nz_2012_indicators.sum())

event         Ironman New Zealand 2012Ironman New Zealand 20...
swim_low                                                    788
swim_cut                                                      0
bike_low                                                   1391
bike_cut                                                      0
run_low                                                    1256
finish_low                                                 1360
finish_cut                                                    0
dtype: object

event         Ironman New Zealand 70.3 2012Ironman New Zeala...
swim_low                                                      0
swim_cut                                                      3
bike_low                                                      0
bike_cut                                                      0
run_low                                                       0
finish_low                                                    0
finish_cut                                                    6
dtype: object

In [34]:
#Remove all of the events with high numbers of bad times.
df_time_removed_events = df_complete_time_data[~df_complete_time_data['event'].isin(events_to_drop)].copy()

events_before_removal = df_complete_time_data.event.nunique()
events_after_removal = df_time_removed_events.event.nunique()

print(f'{events_before_removal - events_after_removal} events removed.')

#Add Ironman NZ 2012 back in
df_time_removed_events = pd.concat([df_time_removed_events, df_nz_2012], ignore_index=True)

274 events removed.


In [35]:
#Now re-run the analysis to see what I'm left with.
df_new_time_indicators = time_test(df_time_removed_events)

print('Number of times flagged per measure. Mostly faster than expected splits, rather than missed cut offs.')
display(df_new_time_indicators[['swim_low', 'swim_cut', 'bike_low', 'bike_cut', 'run_low', 'finish_low', 'finish_cut']].sum())

print('Events with large numbers of times flagged.')
display(df_new_time_indicators.groupby('event').sum().sum(axis=1).sort_values(ascending=False).head(10))

Number of times flagged per measure. Mostly faster than expected splits, rather than missed cut offs.


swim_low       384
swim_cut      1046
bike_low        74
bike_cut       437
run_low        190
finish_low      10
finish_cut    1621
dtype: int64

Events with large numbers of times flagged.


event
Ironman Augusta 70.3 2013                 53
Ironman Oceanside California 70.3 2012    49
Ironman World Championship 2022           37
Ironman Copenhagen 2016                   37
Ironman Timberman 70.3 2010               35
Ironman Texas 70.3 2012                   34
Ironman Oceanside California 70.3 2011    34
Ironman Eagleman 70.3 2013                34
Ironman Mont-Tremblant 70.3 2014          31
Ironman Mont-Tremblant 70.3 2013          31
dtype: int64

In [36]:
#Appears that a large number of half distance events are affected

#Calculation of number of half and full events before any are removed.
number_events_before = df_complete_time_data[['event', 'distance']].drop_duplicates()['distance'].value_counts()

#Calculation after events have been removed.
number_events_after = df_time_removed_events[['event', 'distance']].drop_duplicates()['distance'].value_counts()

print(f'{number_events_after[0]} half distance events kept out of {number_events_before[0]}')
print(f'{number_events_after[1]} full distance events kept out of {number_events_before[1]}')

698 half distance events kept out of 942
447 full distance events kept out of 476


### Removing the remaining records with bad times

Achieve the cleanest possible time data for analysis by removing the remaining bad records. This should be the last action done in cleaning the data, unless anything becomes apparent during the visualisation and analysis.

In [37]:
#Extracting indices of records with a time outside of the accepted ranges.
records_to_remove = df_new_time_indicators[df_new_time_indicators[['swim_low', 'swim_cut', 'bike_low', 'bike_cut', 'run_low', 'finish_low', 'finish_cut']].sum(axis=1).ge(1)].index.tolist()

df_time_data = df_time_removed_events.drop(records_to_remove)

## Exporting the data for analysis

The two separate dataframes need to be exported. This will allow for their use in the analysis programme.

In [38]:
df_all_data.to_csv(cwd+"\data_categorical_analysis.csv")
df_time_data.to_csv(cwd+"\data_numerical_analysis.csv")
df_iron.to_csv(cwd+"\data_ironman.csv") #Added an export of the unclean ironman data. I need a dataframe with all of the events present for plotting the event locations.